<a href="https://colab.research.google.com/github/pairuoyefe/Sales-model/blob/main/Obesity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import matplotlib as plt

In [4]:
data = pd.read_csv(r'Obesity prediction.csv')

In [5]:
data


,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,Obesity
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [6]:
#Cleaning data

data['Obesity'] = data['Obesity'].replace({'Normal_Weight':0,'Overweight_Level_I':1,'Overweight_Level_II':2,'Obesity_Type_I':3,'Obesity_Type_II':4,'Obesity_Type_III':5,'Insufficient_Weight'
:6})
data['MTRANS'] = data['MTRANS'].replace({'Public_Transportation':0,'Automobile':0,'Motorbike':0,'Bike':0,'Walking':1})
data['Gender'] = data['Gender'].replace({'Female':0,'Male':1})
data['SMOKE'] = data['SMOKE'].replace({'no':0,'yes':1})
data['family_history'] = data['family_history'].replace({'no':0,'yes':1})



<ipython-input-6-9a408bb9412a>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Obesity'] = data['Obesity'].replace({'Normal_Weight':0,'Overweight_Level_I':1,'Overweight_Level_II':2,'Obesity_Type_I':3,'Obesity_Type_II':4,'Obesity_Type_III':5,'Insufficient_Weight'
<ipython-input-6-9a408bb9412a>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['MTRANS'] = data['MTRANS'].replace({'Public_Transportation':0,'Automobile':0,'Motorbike':0,'Bike':0,'Walking':1})
<ipython-input-6-9a408bb9412a>:6: FutureWarning: Downcasting behavi

In [7]:
data

,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,Obesity
0,0,21.000000,1.620000,64.000000,1,no,2.0,3.0,Sometimes,0,2.000000,no,0.000000,1.000000,no,0,0
1,0,21.000000,1.520000,56.000000,1,no,3.0,3.0,Sometimes,1,3.000000,yes,3.000000,0.000000,Sometimes,0,0
2,1,23.000000,1.800000,77.000000,1,no,2.0,3.0,Sometimes,0,2.000000,no,2.000000,1.000000,Frequently,0,0
3,1,27.000000,1.800000,87.000000,0,no,3.0,3.0,Sometimes,0,2.000000,no,2.000000,0.000000,Frequently,1,1
4,1,22.000000,1.780000,89.800000,0,no,2.0,1.0,Sometimes,0,2.000000,no,0.000000,0.000000,Sometimes,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,20.976842,1.710730,131.408528,1,yes,3.0,3.0,Sometimes,0,1.728139,no,1.676269,0.906247,Sometimes,0,5
2107,0,21.982942,1.748584,133.742943,1,yes,3.0,3.0,Sometimes,0,2.005130,no,1.341390,0.599270,Sometimes,0,5
2108,0,22.524036,1.752206,133.689352,1,yes,3.0,3.0,Sometimes,0,2.054193,no,1.414209,0.646288,Sometimes,0,5
2109,0,24.361936,1.739450,133.346641,1,yes,3.0,3.0,Sometimes,0,2.852339,no,1.139107,0.586035,Sometimes,0,5


In [8]:
data = data.drop(columns='SCC')

In [11]:
data

,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,FAF,TUE,CALC,MTRANS,Obesity
0,0,21.000000,1.620000,64.000000,1,no,2.0,3.0,Sometimes,0,2.000000,0.000000,1.000000,no,0,0
1,0,21.000000,1.520000,56.000000,1,no,3.0,3.0,Sometimes,1,3.000000,3.000000,0.000000,Sometimes,0,0
2,1,23.000000,1.800000,77.000000,1,no,2.0,3.0,Sometimes,0,2.000000,2.000000,1.000000,Frequently,0,0
3,1,27.000000,1.800000,87.000000,0,no,3.0,3.0,Sometimes,0,2.000000,2.000000,0.000000,Frequently,1,1
4,1,22.000000,1.780000,89.800000,0,no,2.0,1.0,Sometimes,0,2.000000,0.000000,0.000000,Sometimes,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,20.976842,1.710730,131.408528,1,yes,3.0,3.0,Sometimes,0,1.728139,1.676269,0.906247,Sometimes,0,5
2107,0,21.982942,1.748584,133.742943,1,yes,3.0,3.0,Sometimes,0,2.005130,1.341390,0.599270,Sometimes,0,5
2108,0,22.524036,1.752206,133.689352,1,yes,3.0,3.0,Sometimes,0,2.054193,1.414209,0.646288,Sometimes,0,5
2109,0,24.361936,1.739450,133.346641,1,yes,3.0,3.0,Sometimes,0,2.852339,1.139107,0.586035,Sometimes,0,5


In [15]:
data = data.drop(columns='CALC')
data = data.drop(columns='CAEC')
data = data.drop(columns='FAVC')

In [16]:
data

,Gender,Age,Height,Weight,family_history,FCVC,NCP,SMOKE,CH2O,FAF,TUE,MTRANS,Obesity
0,0,21.000000,1.620000,64.000000,1,2.0,3.0,0,2.000000,0.000000,1.000000,0,0
1,0,21.000000,1.520000,56.000000,1,3.0,3.0,1,3.000000,3.000000,0.000000,0,0
2,1,23.000000,1.800000,77.000000,1,2.0,3.0,0,2.000000,2.000000,1.000000,0,0
3,1,27.000000,1.800000,87.000000,0,3.0,3.0,0,2.000000,2.000000,0.000000,1,1
4,1,22.000000,1.780000,89.800000,0,2.0,1.0,0,2.000000,0.000000,0.000000,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,20.976842,1.710730,131.408528,1,3.0,3.0,0,1.728139,1.676269,0.906247,0,5
2107,0,21.982942,1.748584,133.742943,1,3.0,3.0,0,2.005130,1.341390,0.599270,0,5
2108,0,22.524036,1.752206,133.689352,1,3.0,3.0,0,2.054193,1.414209,0.646288,0,5
2109,0,24.361936,1.739450,133.346641,1,3.0,3.0,0,2.852339,1.139107,0.586035,0,5


In [20]:
X = data.drop(columns='Obesity')
y = data['Obesity']

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=43,train_size=0.8)


In [21]:
X

,Gender,Age,Height,Weight,family_history,FCVC,NCP,SMOKE,CH2O,FAF,TUE,MTRANS
0,0,21.000000,1.620000,64.000000,1,2.0,3.0,0,2.000000,0.000000,1.000000,0
1,0,21.000000,1.520000,56.000000,1,3.0,3.0,1,3.000000,3.000000,0.000000,0
2,1,23.000000,1.800000,77.000000,1,2.0,3.0,0,2.000000,2.000000,1.000000,0
3,1,27.000000,1.800000,87.000000,0,3.0,3.0,0,2.000000,2.000000,0.000000,1
4,1,22.000000,1.780000,89.800000,0,2.0,1.0,0,2.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,20.976842,1.710730,131.408528,1,3.0,3.0,0,1.728139,1.676269,0.906247,0
2107,0,21.982942,1.748584,133.742943,1,3.0,3.0,0,2.005130,1.341390,0.599270,0
2108,0,22.524036,1.752206,133.689352,1,3.0,3.0,0,2.054193,1.414209,0.646288,0
2109,0,24.361936,1.739450,133.346641,1,3.0,3.0,0,2.852339,1.139107,0.586035,0


In [22]:
Model1 = RandomForestClassifier()
Model2 = KNeighborsClassifier()

In [23]:
kf = KFold(n_splits=20,random_state=43,shuffle=True)

kfscore1 = cross_val_score(Model1,X_train,y_train,cv=kf)
kfscore2 = cross_val_score(Model2,X_train,y_train,cv=kf)

In [24]:
kfscore1

array([0.96470588, 0.92941176, 0.97647059, 0.96470588, 0.92941176,
       0.97647059, 0.97647059, 0.95294118, 0.92857143, 0.91666667,
       0.98809524, 0.92857143, 0.96428571, 0.95238095, 0.94047619,
       0.91666667, 0.96428571, 0.94047619, 0.92857143, 0.89285714])

In [25]:
kfscore2

array([0.85882353, 0.81176471, 0.85882353, 0.92941176, 0.85882353,
       0.85882353, 0.82352941, 0.81176471, 0.82142857, 0.79761905,
       0.92857143, 0.83333333, 0.9047619 , 0.86904762, 0.79761905,
       0.8452381 , 0.89285714, 0.83333333, 0.83333333, 0.86904762])

In [26]:
param = ({
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt'],
    'bootstrap': [True, False]
})

In [27]:
Grids = RandomizedSearchCV(Model1,param,cv=5, scoring='accuracy',n_jobs=-1)

In [28]:
Grids.fit(X_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [50, 100, 200]},
                   scoring='accuracy')

In [29]:
Grids.best_score_

np.float64(0.9478552490650186)

In [30]:
Grids.best_params_

{'n_estimators': 100,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': False}

In [31]:
y_pred = Grids.best_estimator_.predict(X_test)

In [32]:
y_pred

array([2, 3, 4, 6, 3, 5, 5, 5, 2, 1, 4, 2, 3, 4, 1, 4, 5, 4, 2, 3, 0, 5,
       0, 5, 4, 0, 3, 5, 3, 5, 3, 4, 1, 5, 0, 2, 1, 1, 6, 0, 5, 6, 4, 1,
       4, 5, 2, 4, 4, 5, 6, 6, 5, 2, 4, 6, 2, 2, 4, 1, 3, 0, 2, 1, 5, 2,
       4, 1, 1, 5, 3, 2, 3, 2, 0, 3, 0, 0, 1, 0, 1, 0, 3, 4, 4, 2, 5, 2,
       2, 0, 0, 5, 5, 4, 5, 3, 3, 6, 0, 5, 6, 3, 2, 4, 4, 5, 3, 4, 0, 2,
       4, 5, 5, 0, 2, 0, 0, 5, 4, 0, 3, 4, 4, 0, 5, 6, 2, 3, 1, 0, 2, 2,
       5, 4, 4, 5, 4, 6, 0, 3, 0, 1, 5, 1, 1, 5, 6, 6, 0, 4, 0, 3, 3, 6,
       0, 5, 4, 0, 5, 6, 5, 3, 3, 3, 4, 4, 6, 5, 5, 3, 3, 5, 3, 1, 6, 6,
       0, 4, 3, 6, 4, 5, 3, 3, 5, 4, 2, 1, 4, 6, 4, 4, 3, 2, 6, 5, 2, 1,
       5, 1, 0, 1, 2, 6, 5, 1, 1, 4, 4, 5, 1, 5, 1, 6, 4, 1, 5, 1, 6, 5,
       4, 4, 3, 3, 1, 4, 4, 4, 4, 2, 3, 0, 3, 0, 0, 5, 2, 5, 4, 5, 5, 4,
       0, 5, 3, 5, 1, 4, 2, 4, 1, 2, 6, 4, 1, 0, 4, 3, 1, 6, 0, 6, 3, 4,
       6, 0, 2, 5, 4, 2, 3, 0, 6, 1, 3, 5, 4, 0, 0, 5, 2, 4, 5, 5, 0, 3,
       4, 3, 5, 4, 3, 3, 5, 3, 3, 0, 2, 4, 3, 1, 6,

In [33]:
import pickle

Obesitymodel = Grids.best_estimator_

In [35]:
pickle.dump(Obesitymodel,open('Obesitymodel.pkl','wb'))

In [36]:
model = pickle.load(open('Obesitymodel.pkl','rb'))

In [37]:
model.predict(X_test)

array([2, 3, 4, 6, 3, 5, 5, 5, 2, 1, 4, 2, 3, 4, 1, 4, 5, 4, 2, 3, 0, 5,
       0, 5, 4, 0, 3, 5, 3, 5, 3, 4, 1, 5, 0, 2, 1, 1, 6, 0, 5, 6, 4, 1,
       4, 5, 2, 4, 4, 5, 6, 6, 5, 2, 4, 6, 2, 2, 4, 1, 3, 0, 2, 1, 5, 2,
       4, 1, 1, 5, 3, 2, 3, 2, 0, 3, 0, 0, 1, 0, 1, 0, 3, 4, 4, 2, 5, 2,
       2, 0, 0, 5, 5, 4, 5, 3, 3, 6, 0, 5, 6, 3, 2, 4, 4, 5, 3, 4, 0, 2,
       4, 5, 5, 0, 2, 0, 0, 5, 4, 0, 3, 4, 4, 0, 5, 6, 2, 3, 1, 0, 2, 2,
       5, 4, 4, 5, 4, 6, 0, 3, 0, 1, 5, 1, 1, 5, 6, 6, 0, 4, 0, 3, 3, 6,
       0, 5, 4, 0, 5, 6, 5, 3, 3, 3, 4, 4, 6, 5, 5, 3, 3, 5, 3, 1, 6, 6,
       0, 4, 3, 6, 4, 5, 3, 3, 5, 4, 2, 1, 4, 6, 4, 4, 3, 2, 6, 5, 2, 1,
       5, 1, 0, 1, 2, 6, 5, 1, 1, 4, 4, 5, 1, 5, 1, 6, 4, 1, 5, 1, 6, 5,
       4, 4, 3, 3, 1, 4, 4, 4, 4, 2, 3, 0, 3, 0, 0, 5, 2, 5, 4, 5, 5, 4,
       0, 5, 3, 5, 1, 4, 2, 4, 1, 2, 6, 4, 1, 0, 4, 3, 1, 6, 0, 6, 3, 4,
       6, 0, 2, 5, 4, 2, 3, 0, 6, 1, 3, 5, 4, 0, 0, 5, 2, 4, 5, 5, 0, 3,
       4, 3, 5, 4, 3, 3, 5, 3, 3, 0, 2, 4, 3, 1, 6,

In [38]:
!pip freeze > requirements.txt